$$ ITI \space AI-Pro: \space Intake \space 44 $$
$$ Recommender \space Systems $$
$$ Lab \space no. \space 3 $$

# `01` Import Necessary Libraries

## `i` Default Libraries

In [2]:
import numpy as np
import pandas as pd
from surprise.reader import Reader
from surprise.dataset import Dataset
from surprise.model_selection import train_test_split
from surprise.prediction_algorithms.knns import KNNWithMeans 
from surprise.prediction_algorithms import SVD
import surprise

## `ii` Additional Libraries
Add imports for additional libraries you used throughout the notebook

----------------------------

# `02` Load Data

Load `songsDataset.csv` file into a dataframe

In [3]:
data = pd.read_csv("Data/songsDataset.csv")
data.head()

,userID,songID,rating
0,0,90409,5
1,4,91266,1
2,5,8063,2
3,5,24427,4
4,5,105433,4


--------------------------

# `03` Matrix Factorization using Gradient Descent

Practice for Matrix Factorization Implementation

**Matrix Factorization Mathematical Derivation**

We know that the matrix factorization breaks the rating matrix $R$ into two matrices $U \in \mathbb{R}^{\#users \times K}$ and $M \in \mathbb{R}^{K \times \#items}$ where K represent the latent space dimensionality.

$R$ can then be approximated through the following equation:

$$
\mathbf{R} \approx \mathbf{U} \times \mathbf{M} = \hat{\mathbf{R}}
$$

The error, incorporating the regularization term, is calculated as follows:

$$
e_{ij}^2 = (r_{ij} - \sum_{k=1}^K{u_{ik}m_{kj}})^2 + \frac{\beta}{2} \sum_{k=1}^K{(||U||^2 + ||M||^2)}
$$

In order to be able to use Stochastic Gradient Descent (SGD) to optimize $U$ and $M$, we need to find the partial derivatives of the error function with respect to both $u_{ik}$ and $m_{kj}$. The partial derivatives can be derived as follows:

$$
\frac{\partial}{\partial u_{ik}}e_{ij}^2 = -2(r_{ij} - \hat{r}_{ij})(m_{kj}) + \frac{\beta}{2} \times (2 u_{ik}) = -2 e_{ij} m_{kj} + \beta u_{ik}
$$

$$
\frac{\partial}{\partial m_{ik}}e_{ij}^2 = -2(r_{ij} - \hat{r}_{ij})(u_{ik}) + \frac{\beta}{2} \times (2 m_{kj}) = -2 e_{ij} u_{ik} + \beta m_{kj}
$$

Thus the update rules will be:

$$
u'_{ik} = u_{ik} + \alpha \frac{\partial}{\partial u_{ik}}e_{ij}^2 = u_{ik} - \alpha(-2 e_{ij} m_{kj} + \beta u_{ik} ) = u_{ik} + \alpha(2 e_{ij} m_{kj} - \beta u_{ik} )
$$

$$
m'_{kj} = m_{kj} + \alpha \frac{\partial}{\partial m_{kj}}e_{ij}^2 = m_{kj} - \alpha(-2 e_{ij} u_{ik} + \beta m_{kj} ) = m_{kj} + \alpha(2 e_{ij} u_{ik} - \beta m_{kj} )
$$

## `0` Construct Utility Matrix from the Data

In [4]:
utility_matrix = data.pivot(index='userID', columns='songID', values='rating').fillna(0)
utility_matrix

songID,2263,2726,3785,8063,12709,13859,16548,17029,19299,19670,...,113954,119103,120147,122065,123176,125557,126757,131048,132189,134732
userID,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199976,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
199980,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199988,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## `i` Matrices Initialization

Initialize the two random weights matrices $U$ and $M$ (Try $K=3$)

**Note**: Refer to the next cell for the dimensions of $U$ and $M$

**Hine**: You may use a function from `numpy.random` module (see, [Documentation](https://numpy.org/doc/stable/reference/random/index.html))

In [5]:
no_users = utility_matrix.shape[0]
no_songs = utility_matrix.shape[1]
k = 3

In [6]:
np.random.seed(23)
U = np.random.rand(no_users, k)
M = np.random.rand(k, no_songs)

## `ii` Define a Function to Implement Matrix Factorization

**Function Parameters**:
- `R`: Utility Matrix [of shape: (no. of users, no. of items)]
- `U`: User Latent Features Array [of shape: (no. of users, K)]
- `M`: Items Latent Features Array [of shape: (K, no. of items)]
- `epochs`: No. of training epochs
- `lr`: Learning rate (alpha)
- `beta`: Regularization Parameter

**Function Output**:
- `U`: Optimized User Latent Features Array
- `M`: Optimized Items Latent Features Array

**Main Procedures**:
1. Calculate predicted ratings
2. Calculate MSE Error
3. Calculate gradients
4. Update $U$ and $M$


In [7]:
def matrix_factorization(R: np.ndarray, U: np.ndarray, M: np.ndarray, epochs: int, lr: float, beta: float):
    """
    Function Parameters:
    - `R`: Utility Matrix [of shape: `(no. of users, no. of items)`]
    - `U`: User Latent Features Array [of shape: `(no. of users, K)`]
    - `M`: Items Latent Features Array [of shape: `(K, no. of items)`]
    - `epochs`: No. of training epochs
    - `lr`: Learning rate (alpha)
    - `beta`: Regularization Parameter

    Function Output:
    - `U`: Optimized User Latent Features Array
    - `M`: Optimized Items Latent Features Array
    """

    # Confirm that no. of features is consistent between U and M
    assert U.shape[1] == M.shape[0], f'U and M must have consistent K. Found K={U.shape[1]} for U and K={M.shape[0]} for M'

    # Extract No. of Features (K)
    K = U.shape[1]

    # Define the Epochs loop
    for epoch in range(epochs):
        # Loop over every element in R
        for i in range(no_users): # Loop over each user
            for j in range(no_songs): # Loop over each item
                if R[i][j] != 0: # Only proceed if the current interaction (i, j) is not missing
                    eij = (R[i][j] - (U[i] @ M[:,j])) # Calculate the error in prediction
                    for k in range(K): # Loop over each latent features dimension
                        # Update Rules for both U and M:
                        U[i, k] = U[i, k] + lr * (2 * eij * (M[k, j] - beta * U[i, k]))
                        M[k, j] = M[k, j] + lr * (2 * eij * (U[i, k] - beta * M[k, j]))


        ## Error Calculation ##
        e_last = e if epoch > 0 else 100000000
        e = 0 # Initialize a variable to accumelate the errors
        for i in range(no_users): # Loop over each user
            for j in range(no_songs): # Loop over each item
                if R[i][j] != 0: # Only proceed if the current interaction (i, j) is not missing
                                # since we only calculate the error for interactions having a ground truth value

                    first_part = (R[i][j] - (U[i] @ M[:,j]))**2 # calculate the first part of the error
                    second_part = 0 # Initialize a variable to accumelate the second part of the error

                    for k in range(K): # Loop over each latent features dimension
                        second_part += beta/2 * (np.linalg.norm(U[i])**2 + np.linalg.norm(M[:,j])**2)

                    e += first_part + second_part # accumelate the error to the total error

        print(f'Epoch {epoch+1}/{epochs}: Total Error = {e}')

        if e < 0.001 or e_last-e < 0.001: # Stop if error is so small or improvement is not significant
            break

    return U, M


### `[Bonus]` Vectorized Error Calculation

Can the error calculation part be vectorized to get rid of for loops?

If you would like a challenge, try to redefine the function in the next cell with a vectorized error calculation.

## `iii` Use the Function to to Optimize the $U$ and $V$

In [11]:
U, M = matrix_factorization(R=utility_matrix.values, U=U, M=M, epochs=10, lr=0.001, beta=0.01)

Epoch 1/10: Total Error = 164392.01608698926
Epoch 2/10: Total Error = 155812.07123563671
Epoch 3/10: Total Error = 147901.44814339426
Epoch 4/10: Total Error = 140603.6310917385
Epoch 5/10: Total Error = 133867.07569297997
Epoch 6/10: Total Error = 127644.77737890398
Epoch 7/10: Total Error = 121893.88723146277
Epoch 8/10: Total Error = 116575.35823847649
Epoch 9/10: Total Error = 111653.61536261246
Epoch 10/10: Total Error = 107096.24752131612


## `iv` Recommend top-K Songs

Recommend top-K ($K=5$) songs for user ($userID=199988$)

Note: Make sure to filter songs they already rated before

In [12]:
Predicted = U @ M

In [13]:
predicted_df = pd.DataFrame(data = Predicted , index=utility_matrix.index, columns=utility_matrix.columns)

In [18]:
predicted_df[predicted_df.index == 199988]

songID,2263,2726,3785,8063,12709,13859,16548,17029,19299,19670,...,113954,119103,120147,122065,123176,125557,126757,131048,132189,134732
userID,,,,,,,,,,,,,,,,,,,,,
199988,4.78449,4.023625,4.31621,4.706907,4.051307,4.728691,4.074786,3.905595,4.694102,3.737918,...,4.483545,3.670355,4.266319,4.78136,4.412065,4.763623,4.565861,4.247964,4.682229,4.410509


Top 5 Recommended Items for User 199988:
	- Top 1 Song: 122065 (Predicted Rating: 6.006507760135539)
	- Top 2 Song: 125557 (Predicted Rating: 5.819802641310125)
	- Top 3 Song: 52611 (Predicted Rating: 5.740148229067794)
	- Top 4 Song: 79622 (Predicted Rating: 5.701478248255747)
	- Top 5 Song: 71582 (Predicted Rating: 5.691077461653294)


------------------------

# `04` Matrix Factorization using SVD Algorithm

Practice for using `SVD` algorithm implementation from `scikit surprise` library.

## `i` Prepare the Data

- Load the Data into `surprise` Dataset
- Split data into train and test


In [10]:
df = data

In [11]:
reader = surprise.reader.Reader(rating_scale=(1, 5))

In [12]:
dataset = surprise.dataset.Dataset.load_from_df(df, reader)
dataset

In [13]:
trainset = dataset.build_full_trainset()
trainset

In [14]:
train, test = surprise.model_selection.split.train_test_split(dataset, 0.2, random_state=1234)

In [15]:
test[0]

(38388, 79622, 1.0)

## `ii` Model Initialization

Instantiate two models:
- Model with baselines (biases)
- Model without baselines

**Note**: Use `surprise.prediction_algorithms.matrix_factorization.SVD` (see, [Documentation](https://surprise.readthedocs.io/en/stable/matrix_factorization.html#:~:text=surprise.prediction_algorithms.matrix_factorization.-,SVD,-(n_factors%3D)))

In [25]:
# Biased Model
model_biased = SVD()

In [24]:
# Non-Biased Model
model_not_biased = SVD(biased=False)

## `iii` Fit each Model on Training Data

In [26]:
# Biased Model
model_biased.fit(train)

In [44]:
# Non-Biased Model
model_not_biased.fit(train)

## `iv` Test both Models on the Testing Data

Compare the errors of the two models using multiple error formulas.

**Note**: Refer to `surprise.accuracy` module (see, [Documentation](https://surprise.readthedocs.io/en/stable/accuracy.html))

In [30]:
# Biased Model
predictions1 = model_biased.test(test)

In [33]:
surprise.accuracy.rmse(predictions1)

RMSE: 1.4977


1.4976804432837063

In [45]:
# Non-Biased Model
predictions2 = model_not_biased.test(test)

In [46]:
surprise.accuracy.rmse(predictions2)

RMSE: 2.1094


2.1094108157024722

## `v` Recommend Top $10$ Songs for User $199988$

Is there a difference in recommended songs from the two models?

In [40]:
# Biased Model
user = 199988
list1 = df[df['userID'] != user]['songID'].unique()
pred1=[]
for l in list1:
    pred = model_biased.predict(user, l, verbose=True).est
    pred1.append(pred)
pred1

user: 199988     item: 90409      r_ui = None   est = 4.22   {'was_impossible': False}
user: 199988     item: 91266      r_ui = None   est = 3.20   {'was_impossible': False}
user: 199988     item: 8063       r_ui = None   est = 3.27   {'was_impossible': False}
user: 199988     item: 24427      r_ui = None   est = 4.17   {'was_impossible': False}
user: 199988     item: 105433     r_ui = None   est = 3.63   {'was_impossible': False}
user: 199988     item: 134732     r_ui = None   est = 3.49   {'was_impossible': False}
user: 199988     item: 105421     r_ui = None   est = 2.95   {'was_impossible': False}
user: 199988     item: 19670      r_ui = None   est = 2.69   {'was_impossible': False}
user: 199988     item: 79622      r_ui = None   est = 3.54   {'was_impossible': False}
user: 199988     item: 86341      r_ui = None   est = 4.47   {'was_impossible': False}
user: 199988     item: 131048     r_ui = None   est = 3.71   {'was_impossible': False}
user: 199988     item: 72017      r_ui = No

[4.215809415163655,
 3.204493701195244,
 3.2730916836439623,
 4.172160451464245,
 3.6326035051485857,
 3.493917682017876,
 2.9454373761734796,
 2.6919475566865563,
 3.5428327868775233,
 4.466647022754071,
 3.7135220578666175,
 3.272730953739335,
 2.8988824029666365,
 2.2368750164532982,
 2.872939016471565,
 3.444800248000437,
 3.787213450300929,
 4.265973505967442,
 3.235054368345684,
 4.064544030739343,
 3.5988227161904893,
 2.7214320860769923,
 3.9608149763700506,
 3.3158240717029885,
 3.2357894911274077,
 4.0437487595304535,
 4.02277391304371,
 3.808215373293598,
 3.435562909997493,
 2.900102030003842,
 3.6423464803493193,
 3.8595916515377207,
 4.251630550550765,
 4.635503948449206,
 4.5570388315929105,
 3.842733432792544,
 3.44674747172856,
 4.419659655323265,
 4.4730026437533255,
 3.749753360641343,
 3.0819194737738567,
 3.2281036683365123,
 3.601946758021226,
 3.544629202666832,
 4.201202854985203,
 3.4126234922915817,
 2.846810630538347,
 4.127079026505885,
 3.5840042223474646,


In [41]:
data = {
    'songID': list1,
    'predicted_rating': pred1
}

# Create a DataFrame with two columns
df2 = pd.DataFrame(data)
# Create a DataFrame with one column
df2.head()

,songID,predicted_rating
0,90409,4.215809
1,91266,3.204494
2,8063,3.273092
3,24427,4.172160
4,105433,3.632604


In [42]:
song_predictions_sorted = df2.sort_values(by='predicted_rating' ,ascending=False )
song_predictions_sorted.head(10)

,songID,predicted_rating
33,56660,4.635504
34,13859,4.557039
38,19299,4.473003
9,86341,4.466647
37,2726,4.419660
17,112023,4.265974
32,40712,4.251631
0,90409,4.215809
44,42781,4.201203
3,24427,4.172160


In [47]:
# Non-Biased Model
list1 = df[df['userID'] != user]['songID'].unique()
pred1=[]
for l in list1:
    pred = model_not_biased.predict(user, l, verbose=True).est
    pred1.append(pred)
pred1

user: 199988     item: 90409      r_ui = None   est = 1.07   {'was_impossible': False}
user: 199988     item: 91266      r_ui = None   est = 1.13   {'was_impossible': False}
user: 199988     item: 8063       r_ui = None   est = 1.31   {'was_impossible': False}
user: 199988     item: 24427      r_ui = None   est = 1.00   {'was_impossible': False}
user: 199988     item: 105433     r_ui = None   est = 1.63   {'was_impossible': False}
user: 199988     item: 134732     r_ui = None   est = 1.00   {'was_impossible': False}
user: 199988     item: 105421     r_ui = None   est = 1.00   {'was_impossible': False}
user: 199988     item: 19670      r_ui = None   est = 1.00   {'was_impossible': False}
user: 199988     item: 79622      r_ui = None   est = 1.00   {'was_impossible': False}
user: 199988     item: 86341      r_ui = None   est = 1.00   {'was_impossible': False}
user: 199988     item: 131048     r_ui = None   est = 2.03   {'was_impossible': False}
user: 199988     item: 72017      r_ui = No

[1.068286910395789,
 1.1341069213167205,
 1.3142702292214405,
 1,
 1.62879039881704,
 1,
 1,
 1,
 1,
 1,
 2.0338111064760134,
 1,
 1,
 1.9447869747785194,
 2.6562304932485725,
 1,
 1,
 1.7098023550107195,
 1,
 1,
 1.4310686586435084,
 2.3382620990709224,
 1.7471642166709171,
 2.8826513360370303,
 1,
 1,
 1,
 1,
 1.6060067310371096,
 2.068689678339886,
 1.5630463790865257,
 1.4728122003786794,
 1,
 4.717675261074011,
 1,
 1.3644361024275118,
 1.2733849980030674,
 4.716245455072792,
 1,
 1.4688775941374848,
 1,
 1.6089067469853595,
 1,
 1,
 1.7027669570897075,
 2.6539668921762702,
 2.393462577553421,
 1,
 1.7356056823618338,
 1,
 1,
 1.1743312310249852,
 1.10527693681669,
 1,
 2.482428720827491,
 1]

In [48]:
data = {
    'songID': list1,
    'predicted_rating': pred1
}

# Create a DataFrame with two columns
df2 = pd.DataFrame(data)
# Create a DataFrame with one column
df2.head()

,songID,predicted_rating
0,90409,1.068287
1,91266,1.134107
2,8063,1.314270
3,24427,1.000000
4,105433,1.628790


In [49]:
song_predictions_sorted = df2.sort_values(by='predicted_rating' ,ascending=False )
song_predictions_sorted.head(10)

,songID,predicted_rating
33,56660,4.717675
37,2726,4.716245
23,43267,2.882651
14,55622,2.656230
45,125557,2.653967
54,122065,2.482429
46,45026,2.393463
21,28985,2.338262
29,16548,2.068690
10,131048,2.033811


----------------------------------------------

$$ Wish \space you \space all \space the \space best \space ♡ $$
$$ Abdelrahman \space Eid $$